In [ ]:
!pip install datasets evaluate cohere faiss-cpu

In [ ]:
!pip install clean-text[emoji] nltk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 5.1 MB/s eta 0:00:00
  Created wheel for emoji: filename=emoji-1.7.0-py3-none-any.whl size=171031 sha256=615cd1b56b5cf5ad39c8fa9ea383ef633cf5eb2a623d1f11bfc6b239210f96e3
  Stored in directory: /root/.cache/pip/wheels/bd/22/e5/b69726d5e1a19795ecd3b3e7464b16c0f1d019aa94ff1c8578
Successfully built emoji


In [ ]:
!pip install langchain==0.2.5 faiss-cpu==1.8.0 cohere==5.5.8 langchain-community==0.2.5 rank_bm25==0.2.2 sentence-transformers==3.0.1
!pip install llama-cpp-python==0.2.78  --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu124

Looking in indexes: https://pypi.org/simple, https://abetlen.github.io/llama-cpp-python/whl/cu124


In [ ]:
from datasets import load_dataset

dataset = load_dataset("squad_v2")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
dataset.shape

{'train': (130319, 5), 'validation': (11873, 5)}

In [ ]:
print(dataset["validation"][0])

{'id': '56ddde6b9a695914005b9628', 'title': 'Normans', 'context': 'The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th and 11th centuries gave their name to Normandy, a region in France. They were descended from Norse ("Norman" comes from "Norseman") raiders and pirates from Denmark, Iceland and Norway who, under their leader Rollo, agreed to swear fealty to King Charles III of West Francia. Through generations of assimilation and mixing with the native Frankish and Roman-Gaulish populations, their descendants would gradually merge with the Carolingian-based cultures of West Francia. The distinct cultural and ethnic identity of the Normans emerged initially in the first half of the 10th century, and it continued to evolve over the succeeding centuries.', 'question': 'In what country is Normandy located?', 'answers': {'text': ['France', 'France', 'France', 'France'], 'answer_start': [159, 159, 159, 159]}}


In [ ]:
validation_data = dataset["validation"].select(range(10))

for example in validation_data:
    question = example["question"]
    context = example["context"]
    reference = example["answers"]["text"][0] if example["answers"]["text"] else ""

    print(f"Question: {question}")
    print(f"Answer: {reference}")
    print(f"Context: {context[:100]}...")  # فقط بخشی از متن برای خلاصه‌نمایی
    print("-" * 50)


Question: In what country is Normandy located?
Answer: France
Context: The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th a...
--------------------------------------------------
Question: When were the Normans in Normandy?
Answer: 10th and 11th centuries
Context: The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th a...
--------------------------------------------------
Question: From which countries did the Norse originate?
Answer: Denmark, Iceland and Norway
Context: The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th a...
--------------------------------------------------
Question: Who was the Norse leader?
Answer: Rollo
Context: The Normans (Norman: Nourmands; French: Normands; Latin: Normanni) were the people who in the 10th a...
--------------------------------------------------
Question: What century did the Normans first gain their separat

In [ ]:
from langchain.schema import Document

documents = []

for item in validation_data:
    context = item["context"]
    question = item["question"]
    answer = item["answers"]
    metadata = {"question": question, "answer": answer}
    documents.append(Document(page_content=context, metadata=metadata))


In [ ]:
from cleantext import clean

def clean_context(text):
    return clean(
        text,
        fix_unicode=True,              # تغییر کاراکترهای یونیکد عجیب
        to_ascii=True,                 # حذف کاراکترهای غیر ASCII
        lower=True,                    # کوچیک کردن حروف
        no_line_breaks=True,          # حذف خطوط شکسته
        no_urls=True,                 # حذف URLها
        no_emails=True,               # حذف ایمیل
        no_phone_numbers=True,        # حذف شماره تلفن
        no_numbers=False,             # اگه می‌خوای عددها رو حذف کنی بذار True
        no_punct=False,               # اگه می‌خوای نقطه‌گذاری حذف شه بذار True
        replace_with_url="<URL>",
        replace_with_email="<EMAIL>",
        replace_with_phone_number="<PHONE>",
        replace_with_number="<NUMBER>",
        replace_with_punct="",
        lang="en"
    )


In [ ]:
from transformers import pipeline

summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

def summarize_context(text, max_length=120, min_length=30):
    summary = summarizer(text, max_length=max_length, min_length=min_length, do_sample=False)
    return summary[0]['summary_text']

config.json: 0.00B [00:00, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

Device set to use cuda:0


In [ ]:
def preprocess_context_pipeline(text):
    cleaned = clean_context(text)

    # اگه خیلی بلنده، خلاصه‌ش کن
    if len(cleaned.split()) > 200:
        cleaned = summarize_context(cleaned)

    return cleaned

In [ ]:
cleaned_docs = []

for doc in documents:
    clean_text = preprocess_context_pipeline(doc.page_content)  # تابعی که در پیام قبل دادم
    cleaned_docs.append(Document(page_content=clean_text, metadata=doc.metadata))

In [ ]:
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

# Embedding Model for converting text to numerical representations
embedding_model = HuggingFaceEmbeddings(
    model_name='BAAI/bge-small-en-v1.5'
)

/tmp/ipython-input-1728599694.py:4: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embedding_model = HuggingFaceEmbeddings(


In [ ]:
from langchain.vectorstores import FAISS

# Create a local vector database
db = FAISS.from_documents(documents, embedding_model)


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


### The RAG Prompt


In [ ]:
from langchain import LlamaCpp

# Make sure the model path is correct for your system!
llm = LlamaCpp(
    model_path="Phi-3-mini-4k-instruct-q4.gguf",
    n_gpu_layers=-1,
    max_tokens=500,
    n_ctx=2048,
    seed=42,
    verbose=False
)

In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA

# Create a prompt template
template = """<|user|>
Based ONLY on the information below, answer the following question with the shortest, most direct possible response (ideally a single word or phrase). Do NOT repeat the question, do NOT include explanations, and do NOT say anything other than the answer itself.

Context:
{context}

Question:
{question}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

# RAG Pipeline
rag = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=db.as_retriever(search_kwargs={'k': 2}),
    chain_type_kwargs={
        "prompt": prompt
    },
    verbose=True
)

In [ ]:
rag.invoke('In what country is Normandy located?')



> Entering new RetrievalQA chain...


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



> Finished chain.


{'query': 'In what country is Normandy located?', 'result': ' France'}

In [ ]:
from tqdm import tqdm
validation_data = dataset["validation"]

# فقط ۱۰ نمونه اول رو انتخاب کن
validation_data_small = validation_data.select(range(10))

predictions = []
references = []

for example in tqdm(validation_data_small, desc="Evaluating"):
    question = example["question"]
    # پاسخ درست اول (ground truth)
    reference = example["answers"]["text"][0] if example["answers"]["text"] else ""

    try:
        response = rag.invoke(question)
        # اگر پاسخ دیکشنری بود، نتیجه رو بگیر، وگرنه کل پاسخ
        prediction = response['result'] if isinstance(response, dict) else response
    except Exception as e:
        print(f"Error for question: {question}\n{e}")
        prediction = ""

    predictions.append(prediction)
    references.append(reference)

print("Evaluation finished.")


Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)




> Entering new RetrievalQA chain...


Evaluating:  10%|█         | 1/10 [00:00<00:07,  1.25it/s]


> Finished chain.


> Entering new RetrievalQA chain...


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Evaluating:  20%|██        | 2/10 [00:07<00:31,  3.98s/it]

...


> Finished chain.


> Entering new RetrievalQA chain...


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Evaluating:  90%|█████████ | 9/10 [00:43<00:05,  5.01s/it]


> Finished chain.


> Entering new RetrievalQA chain...


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Evaluating: 100%|██████████| 10/10 [02:54<00:00, 17.44s/it]


> Finished chain.
Evaluation finished.


In [ ]:
import evaluate

squad_metric = evaluate.load("squad")


In [ ]:
# فرض: predictions و references دو لیست متنی هستن
# مثل: predictions = ['Normandy is located in France.'], references = ['France']

# اگه شناسه‌ای نداری می‌تونی idهای ساختگی بدی
formatted_predictions = [
    {"id": str(i), "prediction_text": pred.strip()} for i, pred in enumerate(predictions)
]

formatted_references = [
    {"id": str(i), "answers": {"text": [ref.strip()], "answer_start": [0]}} for i, ref in enumerate(references)
]


In [ ]:
results = squad_metric.compute(
    predictions=formatted_predictions,
    references=formatted_references
)

print(f"F1 Score: {results['f1']:.2f}")
print(f"Exact Match (EM): {results['exact_match']:.2f}")


F1 Score: 44.29
Exact Match (EM): 20.00


In [ ]:
predictions

[' France',
 ' Tenths-eleventh centuries',
 ' Denmark, Iceland, Norway',
 ' Rollo',
 ' First half of the 10th century',
 ' Normans',
 ' Region of\n\nFrance',
 ' Rollo',
 ' Not specified in context.',
 ' William']

In [ ]:
references

['France',
 '10th and 11th centuries',
 'Denmark, Iceland and Norway',
 'Rollo',
 '10th century',
 '',
 '',
 '',
 '',
 'William the Conqueror']

In [ ]:
def recall_at_k(query, answer, k=5):
    retrieved = db.similarity_search(query, k)
    retrieved_texts = [doc.page_content.lower() for doc in retrieved]
    return any(answer.lower() in doc for doc in retrieved_texts)

# تست روی 100 سوال
recall_scores = [recall_at_k(ex["question"], example["answers"]["text"][0] if example["answers"]["text"] else "") for ex in validation_data_small]
recall_at_5 = sum(recall_scores) / len(recall_scores)
print(f"Recall@5: {recall_at_5:.2f}")

Recall@5: 0.10


In [ ]:
from sentence_transformers import SentenceTransformer, util


# بارگذاری مدل
model = SentenceTransformer('all-MiniLM-L6-v2')

# محاسبه شباهت هر جفت و گرفتن میانگین
similarities = []
for ref, pred in zip(references, predictions):
    # اگر هر دو خالی باشن، مشابهت کامل در نظر گرفته میشه
    if ref.strip() == '' and pred.strip() == '':
        sim = 1.0
    # اگر یکی‌شون خالیه، شباهت صفر
    elif ref.strip() == '' or pred.strip() == '':
        sim = 0.0
    else:
        sim = util.cos_sim(model.encode(ref), model.encode(pred))[0][0].item()

    similarities.append(sim)

# میانگین شباهت‌ها
average_similarity = sum(similarities) / len(similarities)
print(f"Semantic Similarity (Average): {average_similarity:.4f}")


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


Semantic Similarity (Average): 0.5496


# "Not answerable based on context."

In [ ]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA

# Create a prompt template
template = """<|user|>
Based ONLY on the information below, answer the following question with the shortest, most direct possible response (ideally a single word or phrase). Do NOT repeat the question, do NOT include explanations, and do NOT say anything other than the answer itself.If the answer is not found in the context, respond with exactly: "Not present in context"

Context:
{context}

Question:
{question}<|end|>
<|assistant|>"""

prompt = PromptTemplate(
    template=template,
    input_variables=["context", "question"]
)

# RAG Pipeline
rag = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type='stuff',
    retriever=db.as_retriever(search_kwargs={'k': 2}),
    chain_type_kwargs={
        "prompt": prompt
    },
    verbose=True
)

In [ ]:
rag.invoke('In what country is Normandy located?')



> Entering new RetrievalQA chain...


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



> Finished chain.


{'query': 'In what country is Normandy located?', 'result': ' France'}

In [ ]:
rag.invoke('When did the Frankish identity emerge?')



> Entering new RetrievalQA chain...


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)



> Finished chain.


{'query': 'When did the Frankish identity emerge?',
 'result': ' Not present in context'}

In [ ]:
from tqdm import tqdm
validation_data = dataset["validation"]

# فقط ۱۰ نمونه اول رو انتخاب کن
validation_data_small = validation_data.select(range(10))

predictions = []
references = []

for example in tqdm(validation_data_small, desc="Evaluating"):
    question = example["question"]
    # پاسخ درست اول (ground truth)
    reference = example["answers"]["text"][0] if example["answers"]["text"] else ""

    try:
        response = rag.invoke(question)
        # اگر پاسخ دیکشنری بود، نتیجه رو بگیر، وگرنه کل پاسخ
        prediction = response['result'] if isinstance(response, dict) else response
    except Exception as e:
        print(f"Error for question: {question}\n{e}")
        prediction = ""

    predictions.append(prediction)
    references.append(reference)

print("Evaluation finished.")


Evaluating:   0%|          | 0/10 [00:00<?, ?it/s]



> Entering new RetrievalQA chain...


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Evaluating:  10%|█         | 1/10 [00:02<00:24,  2.74s/it]


> Finished chain.


> Entering new RetrievalQA chain...


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Evaluating:  20%|██        | 2/10 [00:08<00:36,  4.57s/it]

...


> Finished chain.


> Entering new RetrievalQA chain...


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Evaluating:  90%|█████████ | 9/10 [00:41<00:04,  4.42s/it]


> Finished chain.


> Entering new RetrievalQA chain...


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)
Evaluating: 100%|██████████| 10/10 [02:49<00:00, 16.99s/it]


> Finished chain.
Evaluation finished.


In [ ]:
import evaluate

squad_metric = evaluate.load("squad")

In [ ]:
def normalize_prediction(pred):
    pred = pred.strip().lower()
    if pred in ["not present in context", "not found in context", "not in context"]:
        return ""  # چون رفرنس هم هیچی نیست
    return pred
predictions = [normalize_prediction(p) for p in predictions]

In [ ]:
formatted_predictions = [
    {"id": str(i), "prediction_text": pred.strip()} for i, pred in enumerate(predictions)
]

formatted_references = [
    {"id": str(i), "answers": {"text": [ref.strip()], "answer_start": [0]}} for i, ref in enumerate(references)
]

In [ ]:
results = squad_metric.compute(
    predictions=formatted_predictions,
    references=formatted_references
)

print(f"F1 Score: {results['f1']:.2f}")
print(f"Exact Match (EM): {results['exact_match']:.2f}")

F1 Score: 51.90
Exact Match (EM): 60.00


In [ ]:
predictions

['france',
 '10th-11th centuries',
 'denmark, iceland, norway',
 'rollo',
 '10th century',
 'normans',
 '',
 'rollo',
 '',
 'william the conqueror']

In [ ]:
references

['France',
 '10th and 11th centuries',
 'Denmark, Iceland and Norway',
 'Rollo',
 '10th century',
 '',
 '',
 '',
 '',
 'William the Conqueror']

In [ ]:
def recall_at_k(query, answer, k=5):
    retrieved = db.similarity_search(query, k)
    retrieved_texts = [doc.page_content.lower() for doc in retrieved]
    return any(answer.lower() in doc for doc in retrieved_texts)

# تست روی 100 سوال
recall_scores = [recall_at_k(ex["question"], example["answers"]["text"][0] if example["answers"]["text"] else "") for ex in validation_data_small]
recall_at_5 = sum(recall_scores) / len(recall_scores)
print(f"Recall@5: {recall_at_5:.2f}")

Recall@5: 0.10


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/module.py:1750: FutureWarning: `encoder_attention_mask` is deprecated and will be removed in version 4.55.0 for `BertSdpaSelfAttention.forward`.
  return forward_call(*args, **kwargs)


In [ ]:
from sentence_transformers import SentenceTransformer, util


# بارگذاری مدل
model = SentenceTransformer('all-MiniLM-L6-v2')

# محاسبه شباهت هر جفت و گرفتن میانگین
similarities = []
for ref, pred in zip(references, predictions):
    # اگر هر دو خالی باشن، مشابهت کامل در نظر گرفته میشه
    if ref.strip() == '' and pred.strip() == '':
        sim = 1.0
    # اگر یکی‌شون خالیه، شباهت صفر
    elif ref.strip() == '' or pred.strip() == '':
        sim = 0.0
    else:
        sim = util.cos_sim(model.encode(ref), model.encode(pred))[0][0].item()

    similarities.append(sim)

# میانگین شباهت‌ها
average_similarity = sum(similarities) / len(similarities)
print(f"Semantic Similarity (Average): {average_similarity:.4f}")


Semantic Similarity (Average): 0.7953
